<a href="https://colab.research.google.com/github/Adhul-R/ml-projects/blob/main/Earthquake_Prediciton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Load and prepare the data
data = pd.read_csv('/content/Earthquakes_Dataset.csv')

# Create label column
data['label'] = data['mag'].apply(lambda x: 1 if x >= 4.0 else 0)

# Drop columns that won’t help directly
data = data.drop(columns=['place', 'time'])

# Encode categorical columns
cat_cols = ['magType', 'type', 'net']
encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))
    encoders[col] = le  # save for inverse_transform later

# Features and target
X = data.drop(columns=['label'])
y = data['label']

# Split for testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# LightGBM parameters tuned to avoid overfitting
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'binary_logloss',
    'learning_rate': 0.05,
    'max_depth': 6,
    'num_leaves': 20,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'bagging_freq': 3,
    'lambda_l1': 1.0,
    'lambda_l2': 1.0,
    'verbose': -1,
    'random_state': 42
}

# Train model using early stopping and validation set
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test)

model = lgb.train(
    params,
    train_data,
    num_boost_round=300,
    valid_sets=[valid_data],
    callbacks=[lgb.early_stopping(30), lgb.log_evaluation(0)]
)

# Predict
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\n Model Accuracy: {accuracy * 100:.2f}%")
print(" Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[285]	valid_0's binary_logloss: 0.000533579

 Model Accuracy: 99.96%
 Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3938
           1       1.00      1.00      1.00       610

    accuracy                           1.00      4548
   macro avg       1.00      1.00      1.00      4548
weighted avg       1.00      1.00      1.00      4548

Confusion Matrix:
 [[3936    2]
 [   0  610]]


In [16]:
# 📥 Example Input (Test Case)
test_input = {
    'mag': 2.56,
    'magType': 'md',
    'type': 'earthquake',
    'longitude': -125.6143341,
    'latitude': 40.21649933,
    'depth_km': 4.95,
    'sig': 101,
    'net': 'nc',
    'nst': 14,
    'dmin': 0.976,
    'rms': 0.12,
    'gap': 325
}

# 🧪 Run prediction
print("\n📌 Prediction on Test Input:")
print(predict_earthquake(test_input))


📌 Prediction on Test Input:
✅ No major earthquake expected.


In [17]:
test_input = {
    'mag': 5.6,                         # High magnitude (≥ 4.0)
    'magType': 'mb',                   # Common magType (change as per dataset)
    'type': 'earthquake',             # Typical type
    'longitude': 93.45,
    'latitude': 27.98,
    'depth_km': 10.0,                 # Shallow depth, often more dangerous
    'sig': 600,                        # High significance
    'net': 'us',                       # USGS network, change to match training data
    'nst': 45,                         # Number of stations
    'dmin': 0.3,                       # Closer to station
    'rms': 0.45,                       # Root mean square of signal
    'gap': 75                          # Azimuthal gap
}
# 🧪 Run prediction
print("\n📌 Prediction on Test Input:")
print(predict_earthquake(test_input))



📌 Prediction on Test Input:
⚠️ Earthquake likely!
